In [1]:
import torch

In [2]:
torch.cuda.device_count()

8

In [ ]:
# 어떤 데이터들을 어떤 지피유에 올릴지 선택해줘야 하기 때문에 Distributed Sampler를 적용해준다 
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_set,
                        rank=rank, num_replicas=args.world_size, shuffle=True)

    valid_sampler = torch.utils.data.distributed.DistributedSampler(valid_set,
                        rank=rank, num_replicas=args.world_size, shuffle=True)
    
    # 모든 지피유들이 동일한 메모리를 사용하게 하고 싶다면 pin memory = True 로.. 
    train_loader = data.DataLoader(train_set, batch_size=int(args.batch_size),
                                    num_workers=args.num_workers, shuffle=False,
                                    pin_memory=True  , sampler=train_sampler)

    valid_loader = data.DataLoader(valid_set, batch_size=int(args.batch_size),
                                    num_workers=args.num_workers, shuffle=False,
                                    pin_memory=True  , sampler=valid_samplerse,)

In [ ]:
    # loss
    # Custom loss 
    #criterion_bce = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1])).cuda(rank)
    criterion_reject = reject_loss.RejectLoss().cuda(rank)

    # optim
    optimizer = select_optimizer(args.optim_function, args.initial_lr, net)

    net = net.cuda(rank)
    net = torch.nn.SyncBatchNorm.convert_sync_batchnorm(net)
    net = DDP(net, device_ids=[rank])

In [ ]:
def run_model(model, world_size) :
    print(args.work_dir, args.exp)
    work_dir = os.path.join(args.work_dir, str(args.exp))

    if not os.path.exists(work_dir):
        os.makedirs(work_dir)


    shutil.copy(__file__, os.path.join(work_dir, 'main.py'))
    
    mp.spawn(main,
            args=(world_size,),
            nprocs=world_size,
            join=True)

def reduce_tensor(tensor, world_size):
    rt = tensor.clone()
    dist.all_reduce(rt, op=dist.ReduceOp.SUM)
    rt /= world_size
    return rt

def init_process(rank, world_size, backend='nccl', ip=1):
    if ip == 1 :
        os.environ['MASTER_ADDR'] = '127.0.0.1'
        os.environ['MASTER_PORT'] = '8888'
    elif ip == 0 : 
        os.environ['MASTER_ADDR'] = '127.0.0.1'
        os.environ['MASTER_PORT'] = '9999'
    dist.init_process_group(backend, rank=rank, world_size=world_size)


'01000'